In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_datasets as tfds
import IPython.display as disp

!pip install geemap
import geemap



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 5.5 MB/s eta 0:00:00


In [2]:
import ee
ee.Authenticate()
ee.Initialize(project="federicodeguio")

In [3]:
geoJSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              13.669909554672131,
              38.44375747598076
            ],
            [
              12.31929587228467,
              38.367171932178536
            ],
            [
              12.130986285383386,
              37.74510712599766
            ],
            [
              12.769761887711041,
              37.3500504756916
            ],
            [
              14.022559100396109,
              36.860128123644
            ],
            [
              15.42085122732766,
              36.456383842077386
            ],
            [
              16.00809874131619,
              37.1031836930147
            ],
            [
              15.863650522255057,
              37.90168784726373
            ],
            [
              14.97123663823433,
              38.43965372078583
            ],
            [
              13.669909554672131,
              38.44375747598076
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}

coords = geoJSON['features'][0]['geometry']['coordinates']
roi = ee.Geometry.Polygon(coords)

In [4]:

collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
                .filterBounds(roi) \
                .filterDate('2022-01-01', '2023-04-10') \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',10)) \
                .sort("CLOUDY_PIXEL_PERCENTAGE")
                #.select('B2','B3','B4')


print(collection.size().getInfo())
mosaic = collection.median()


895


In [5]:
# collection.aggregate_array('CLOUDY_PIXEL_PERCENTAGE').getInfo()


In [6]:
imageList = collection.toList(collection.size())

image = imageList.get(0)
image = ee.Image(image)

In [7]:
Map = geemap.Map()
vis_param = {'min': 0,
             'max': 3000,
             'bands': ['B8', 'B4', 'B2'],
             'gamma': 1.5}

Map.addLayer(mosaic, vis_param, "First image")
Map.centerObject(roi, 8)

Map

Map(center=[37.61669053728234, 14.244243908746563], controls=(WidgetControl(options=['position', 'transparent_…